In [ ]:
pip install split-folders

In [ ]:
import kagglehub
import os
import splitfolders
import tensorflow as tf, numpy as np
from collections import Counter
from tensorflow.keras.applications.efficientnet import preprocess_input
from pathlib import Path
from collections import Counter
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import load_model



In [ ]:
# Download latest version
path = kagglehub.dataset_download("feyzazkefe/trashnet")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'trashnet' dataset.
Path to dataset files: /kaggle/input/trashnet


In [ ]:
sub_path = os.path.join(path, "dataset-resized")
subfolders = [f for f in os.listdir(sub_path) if os.path.isdir(os.path.join(sub_path, f))]
print(subfolders)

['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']


In [ ]:
base_path = os.path.join(path, "dataset-resized")
output_dir = "/content/Split"

# ratio can be adjusted, e.g. (0.8, 0.1, 0.1)
splitfolders.ratio(base_path, output=output_dir, seed=1337, ratio=(.8, .1, .1))

Copying files: 2527 files [00:13, 194.28 files/s]


In [ ]:
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")
test_dir  = os.path.join(output_dir, "test")

In [ ]:
SEED = 42
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
NUM_CLASSES = 2 # Ensure this is correctly set if not already


train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, seed=SEED, shuffle=True
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, seed=SEED, shuffle=False
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=False
)

def preprocess(image, label):
    image = preprocess_input(image)
    label = tf.one_hot(label, NUM_CLASSES) # One-hot encode labels
    return image, label

train_ds = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
test_ds = test_ds.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

Found 2019 files belonging to 6 classes.
Found 251 files belonging to 6 classes.
Found 257 files belonging to 6 classes.


In [ ]:
CFG = {
    "dataset_root": "/content/Split",   # change per dataset
    "img_size": (224, 224),                  # EfficientNetB0 likes 224; B3 likes 300; set once
    "batch_size": 32,
    "seed": 42,
    "backbone": "efficientnetb0",            # "efficientnetb0"|"efficientnetb3"|"mobilenetv3large"
    "use_mixed_precision": True
}

def make_splits(root):
    root = Path(root)
    has_val = (root/"val").exists()
    train_ds = tf.keras.utils.image_dataset_from_directory(
            root/"train", image_size=CFG["img_size"], batch_size=CFG["batch_size"],
            seed=CFG["seed"], shuffle=True
        )
    val_ds = tf.keras.utils.image_dataset_from_directory(
            root/"val", image_size=CFG["img_size"], batch_size=CFG["batch_size"],
            seed=CFG["seed"], shuffle=False
        )
    test_ds = tf.keras.utils.image_dataset_from_directory(
            root/"test", image_size=CFG["img_size"], batch_size=CFG["batch_size"],
            seed=CFG["seed"], shuffle=False
        )
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = make_splits(CFG["dataset_root"])
CLASS_NAMES = train_ds.class_names
NUM_CLASSES = len(CLASS_NAMES)
print("Classes:", CLASS_NAMES)

Found 2019 files belonging to 6 classes.
Found 251 files belonging to 6 classes.
Found 257 files belonging to 6 classes.
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.05),
])

# Use the matching preprocess for your backbone
if CFG["backbone"].startswith("efficientnet"):
    from tensorflow.keras.applications.efficientnet import preprocess_input
elif CFG["backbone"].startswith("mobilenetv3"):
    from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
else:
    preprocess_input = lambda x: x

def prep(ds, training=False):
    ds = ds.map(lambda x,y: (tf.cast(x, tf.float32), y), num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.map(lambda x,y: (aug(x, training=True), y), num_parallel_calls=AUTOTUNE)
    ds = ds.map(lambda x,y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
    return ds.cache().prefetch(AUTOTUNE)

train_ds = prep(train_ds, training=True)
val_ds   = prep(val_ds,   training=False)
test_ds  = prep(test_ds,  training=False) if test_ds is not None else None

In [ ]:
counts = Counter(int(y.numpy()) for _, y in train_ds.unbatch().take(10**9))
total = sum(counts.values())
class_weight = {c: total/(NUM_CLASSES*counts[c]) for c in range(NUM_CLASSES)}
print("class_weight:", class_weight)

class_weight: {0: 1.045031055900621, 1: 0.84125, 2: 1.0259146341463414, 3: 0.708421052631579, 4: 0.874025974025974, 5: 3.0871559633027523}


In [ ]:
if CFG["use_mixed_precision"]:
    tf.keras.mixed_precision.set_global_policy("mixed_float16")

from tensorflow.keras import layers as L, Model
from tensorflow.keras.optimizers import Adam

IMG_SHAPE = CFG["img_size"] + (3,)

def get_backbone(name):
    name = name.lower()
    if name == "efficientnetb0":
        return tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=IMG_SHAPE)
    if name == "efficientnetb3":
        return tf.keras.applications.EfficientNetB3(include_top=False, weights="imagenet", input_shape=IMG_SHAPE)
    if name == "mobilenetv3large":
        return tf.keras.applications.MobileNetV3Large(include_top=False, weights="imagenet", input_shape=IMG_SHAPE)
    raise ValueError("Unknown backbone")

base = get_backbone(CFG["backbone"])
base.trainable = False

inp = L.Input(shape=IMG_SHAPE)
x = base(inp, training=False)
x = L.GlobalAveragePooling2D()(x)
x = L.Dropout(0.3)(x)

if NUM_CLASSES == 2:
    out = L.Dense(1, activation="sigmoid", dtype="float32")(x)
    loss = tf.keras.losses.BinaryCrossentropy()
    model_metrics = [
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.AUC(name="pr_auc", curve="PR")
    ]
else:
    out = L.Dense(NUM_CLASSES, activation="softmax", dtype="float32")(x)
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    model_metrics = ["accuracy"]

model = Model(inp, out)
model.compile(optimizer=Adam(1e-3), loss=loss, metrics=model_metrics)
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │         7,686 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,057,257 (15.48 MB)

 Trainable params: 7,686 (30.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
monitor_key = "val_pr_auc" if NUM_CLASSES == 2 else "val_accuracy"
ckpt = ModelCheckpoint("best_head.keras", monitor=monitor_key, mode="max", save_best_only=True, verbose=1)
es   = EarlyStopping(monitor=monitor_key, mode="max", patience=5, restore_best_weights=True, verbose=1)
rlr  = ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, verbose=1)

hist1 = model.fit(
    train_ds, validation_data=val_ds, epochs=8,
    class_weight=class_weight, callbacks=[ckpt, es, rlr]
)


Epoch 1/8
20/64 ━━━━━━━━━━━━━━━━━━━━ 8:20 11s/step - accuracy: 0.2665 - loss: 1.7265

KeyboardInterrupt: 

In [ ]:
model.save("/content/garbage_classifier_Ver4_Frozen.keras")

from google.colab import files
files.download("garbage_classifier_Ver4_Frozen.keras")

In [ ]:
model_path = "/content/garbage_classifier_Ver4_Frozen.keras"
model = keras.models.load_model(model_path)

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 8 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 4 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Fine-tune
for layer in base.layers[-60:]:                       # adjust depth
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

model.compile(optimizer=Adam(2e-5), loss=loss, metrics=model_metrics)


In [ ]:
ckpt_ft = ModelCheckpoint("best_finetune.keras", monitor=monitor_key, mode="max", save_best_only=True, verbose=1)
es_ft   = EarlyStopping(monitor=monitor_key, mode="max", patience=5, restore_best_weights=True, verbose=1)
rlr_ft  = ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, verbose=1)

hist2 = model.fit(
    train_ds, validation_data=val_ds, epochs=12,
    class_weight=class_weight, callbacks=[ckpt_ft, es_ft, rlr_ft]
)

Epoch 1/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.8764 - loss: 0.3840 
Epoch 1: val_accuracy improved from -inf to 0.84861, saving model to best_finetune.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 784s 12s/step - accuracy: 0.8763 - loss: 0.3843 - val_accuracy: 0.8486 - val_loss: 0.5047 - learning_rate: 1.8000e-06
Epoch 2/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.8646 - loss: 0.3965 
Epoch 2: val_accuracy did not improve from 0.84861
64/64 ━━━━━━━━━━━━━━━━━━━━ 798s 12s/step - accuracy: 0.8646 - loss: 0.3966 - val_accuracy: 0.8486 - val_loss: 0.5047 - learning_rate: 1.8000e-06
Epoch 3/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.8786 - loss: 0.4010 
Epoch 3: val_accuracy did not improve from 0.84861

Epoch 3: ReduceLROnPlateau reducing learning rate to 5.399999849942105e-07.
64/64 ━━━━━━━━━━━━━━━━━━━━ 806s 12s/step - accuracy: 0.8787 - loss: 0.4010 - val_accuracy: 0.8486 - val_loss: 0.5047 - learning_rate: 1.8000e-06
Epoch 4/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1

In [ ]:
model.save("/content/garbage_classifier_Ver4_60Unfrozen.keras")

from google.colab import files
files.download("garbage_classifier_Ver4_60Unfrozen.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>